In [2]:
#Import needed libraries
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import csv
import datetime
import os,platform
from scipy.optimize import curve_fit
import scipy
from scipy.fft import fft, rfft, irfft,ifft,fftfreq
from scipy.signal import argrelextrema
from scipy.stats import norm
from lmfit.models import SkewedGaussianModel
import re
import sqlite3

# Please update these path varibles to your local machine.
env_session_file_location = r"\\172.24.54.234\\NAS-Lab28\\Data"
NAS_Path = r"\\172.24.54.234\\NAS-Lab28"
Plot_path= r"\\172.24.54.234\\NAS-Lab28\\Users\\Bane\\Plots"


# Session Analysis

## Build Run list from new database

In [ ]:
Connection = sqlite3.connect(NAS_Path+"\\Data\\lab_28_run.db")
RunDatabase = pd.read_sql("SELECT * from runs", Connection).sort_values(by='run', ascending=False)




'Laser power measurments'

In [14]:
rn = 500
RunInfo = RunDatabase.query(f'run == {rn}').iloc[0]


session_file_name= f"{env_session_file_location}\\\{RunInfo['session']:03}_{RunInfo['date']}.h5"
print(session_file_name)
RunInfo

\\172.24.54.234\\NAS-Lab28\\Data\\024_241030.h5


run                                                    500
old_run_name                                20241030-00295
session                                                 24
date                                                241030
configuration                                           32
grid                                                   2.0
run_type                                     commissioning
cathode_V                                              0.0
cathode_V_err                                          0.5
cathode_grid_V                                         0.0
cathode_grid_V_err                                     0.5
anode_grid_V           b'\x00\x00\x00\x00\x00\x00\x00\x00'
anode_grid_V_err                                       0.5
anode_V                                                0.0
anode_V_err                                            0.5
laser_A                                               27.5
laser_A_err                                           0.

In [ ]:
session_files = glob.glob(env_session_file_location + "\\*.h5")

for session_file in session_files:
    # Open the HDF5 file
    with h5py.File(session_file, 'r') as f:
        # Check if 'data' exists in the file
        subdir = f['run']
        for runnumber in subdir.keys():
            channels=subdir[runnumber].keys()
            waveform_data={}
            for channel in channels:
                waveform_data[channel] = subdir[runnumber][channel]
                print(subdir[runnumber][channel])

                print(subdir[runnumber][channel])
                print(waveform_data)
            break


    break


<HDF5 dataset "anode": shape (12501,), type "<f4">


AttributeError: 'Dataset' object has no attribute 'values'

## Laser power analysis - calibration for laser settings to injected power


In [ ]:
runnumber = 500
debug = 10

#def MeasureLaserpower(runnumber=0, debug=0):
if 1==1:

    if debug >=10:
        print(f"Laser power analysis for Run number: {runnumber}")
    Runinfo = RunDatabase.query(f'run == {runnumber}').iloc[0]
    

    
